<a href="https://colab.research.google.com/github/erikcferreira/Bitcoin_Fear_Greed_Analysis/blob/main/Bitcoin_Fear_Greed_HeatBar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# --- Setup ---

In [13]:
!pip install yfinance plotly streamlit

In [14]:
import pandas as pd
import yfinance as yf
import requests
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Extract & Transform Function --- #

In [15]:
# Take a look into the data.
btc = yf.Ticker("BTC-USD")
df = btc.history(period="max")
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,0.0,0.0
2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,0.0,0.0
2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,0.0,0.0
2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,0.0,0.0
2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,0.0,0.0


In [16]:
def get_bitcoin_data():
  """Fetches BTC-USD price history."""
  print("Fetching Bitcoin prices...")
  btc = yf.Ticker("BTC-USD")
  df = btc.history(period="max")
  # Reset index to make Date a column, and remove timezone.
  df = df.reset_index()
  # Pandas .dt methods gives access to datetime-specific properties and methods;
  # .tz_localize(None) Removes timezone information from datetime values.
  df['Data'] = df ['Date'].dt.tz_localize(None)
  return df[['Date','Close']]

In [17]:
btc_df = get_bitcoin_data()
print(btc_df.tail())

Fetching Bitcoin prices...
                          Date         Close
4138 2026-01-15 00:00:00+00:00  95551.187500
4139 2026-01-16 00:00:00+00:00  95525.117188
4140 2026-01-17 00:00:00+00:00  95099.921875
4141 2026-01-18 00:00:00+00:00  93634.429688
4142 2026-01-19 00:00:00+00:00  92944.164062


In [18]:
#Take a look into the data.
url = "https://api.alternative.me/fng/?limit=0"
r = requests.get(url)
data = r.json()['data']
df = pd.DataFrame(data)
print(df.head())
print(df.dtypes)

  value value_classification   timestamp time_until_update
0    44                 Fear  1768780800              5237
1    49              Neutral  1768694400               NaN
2    50              Neutral  1768608000               NaN
3    49              Neutral  1768521600               NaN
4    61                Greed  1768435200               NaN
value                   object
value_classification    object
timestamp               object
time_until_update       object
dtype: object


In [19]:
def get_fear_greed_data():
  """Fetches Fear & Greed Index from Alternative.me."""
  print("Fetching Fear & Greed Index...")
  url = "https://api.alternative.me/fng/?limit=0"
  r = requests.get(url)
  data = r.json()['data']

  df = pd.DataFrame(data)
  # Convert Unix timestamp to date
  df['Date'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
  # Convert original object 'value' column to int.
  df['value'] = df['value'].astype(int)

  # Sort the date chronologically.
  df = df.sort_values(by='Date')

  return df[['Date', 'value', 'value_classification']]


In [20]:
def merge_data(btcdf, fg_df):
  """Merges the two datasets on Date."""
  # Used inner merger to keep only data present in both price and sentiment (time).
  # Convert timezone info to avoid conflict in the merge
  if btc_df['Date'].dt.tz is not None:
    btc_df['Date'] = btc_df['Date'].dt.tz_localize(None)

  if fg_df['Date'].dt.tz is not None:
    fg_df['Date'] = fg_df['Date'].dt.tz_localize(None)

  merged_df = pd.merge(btc_df, fg_df, on='Date', how='inner')
  return merged_df

In [21]:
#Load the data and merge the dataframes

btc_df = get_bitcoin_data()
fg_df = get_fear_greed_data()
merged_df = merge_data(btc_df, fg_df)

print(f"Data Loaded! Rows: {len(df)}, take a look on it.")
print(merged_df.head())
print(merged_df.tail())

Fetching Bitcoin prices...
Fetching Fear & Greed Index...
Data Loaded! Rows: 2906, take a look on it.
        Date        Close  value value_classification
0 2018-02-01  9170.540039     30                 Fear
1 2018-02-02  8830.750000     15         Extreme Fear
2 2018-02-03  9174.910156     40                 Fear
3 2018-02-04  8277.009766     24         Extreme Fear
4 2018-02-05  6955.270020     11         Extreme Fear
           Date         Close  value value_classification
2901 2026-01-15  95551.187500     61                Greed
2902 2026-01-16  95525.117188     49              Neutral
2903 2026-01-17  95099.921875     50              Neutral
2904 2026-01-18  93634.429688     49              Neutral
2905 2026-01-19  92944.164062     44                 Fear


# --- Creating the Visualization ---

In [22]:
def fear_greed_color(value):
  """Defining values to fear & greed colors (0 to 100)"""
  if value <= 25: return '#FF0000' #RED
  elif value < 46: return '#FF6347' #LIGHT RED
  elif value <= 54: return '#808080' #GREY
  elif value < 75: return '#90EE90' #LIGHT GREEN
  else: return '#008000' # GREEN

In [33]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=merged_df['Date'],
        y=merged_df['Close'],
        mode='markers',
        name='Bitcoin Price',
        marker=dict(
            size=1,
            color=merged_df['value'], # The value that drives the color
            cmin=0, cmax=100,  # Lock the scale: 0 is Min, 100 is Max

            # THE GRADIENT: Red (0) -> Yellow (0.5) -> Green (1)
            colorscale=[
                [0.0, 'rgb(255, 0, 0)'],      # Red (Fear)
                [0.5, 'rgb(255, 255, 0)'],    # Yellow (Neutral)
                [1.0, 'rgb(0, 255, 0)']       # Green (Greed)
            ],

            # THE RIGHT AXIS (Color Bar)
            showscale=True,
            colorbar=dict(
                title="Fear & Greed Index",
                titleside="right",
                thickness=15
            )
        )
    )
)

In [34]:
fig.update_layout(
    title='<b>Bitcoin Price and Fear Index (2018-2025)</b>',
    template="plotly_dark",
    height=600,
    legend=dict(orientation="h", y=1.1, x=0.5, xanchor="center"),
    margin=dict(l=20, r=20, t=60, b=20),
    hovermode="x unified" # This makes the tooltip show BOTH values at once!
)

In [35]:
# Configure Left Axis (Price)
fig.update_yaxes(
    title_text="Bitcoin Price (USD) - Log Scale",
    type="log", # Change scale into Log scale
    showgrid=True,
    gridcolor='rgba(128, 128, 128, 0.2)'
)

# Configure X Axis
fig.update_xaxes(
    title_text="Date",
    showgrid=False
)

fig.show()